### MASTER-THESIS: Constructing a Knowledge Graph by extracting information from financial news articles

#### IMPORT LIBRARIES

In [1]:
# Note: Import modules
import torch
import time
import subprocess
from random import choices

In [2]:
# Note: Import own scripts and classes
from src.settings.enums import ExtractionType, SpacyTask
from src.A_data.data_loader import DataLoader
from main_process import SpacyProcess, Process

#### Start docker container

##### Due to dependency conflicts, some pipe components must be isolated in docker containers:
    - Coreference resolution with crosslingual-coreference module (Image: ghcr.io/rainergo/img-xx-coref)
    - Coreference with Generative LLMs (Image: ghcr.io/rainergo/img-llm-extract-coref)
    - Topic Modelling with Generative LLMs (Image: ghcr.io/rainergo/img-llm-extract-topic)
    
These docker images are public and will be downloaded from my Github image repository once the

    - docker compose up
   
command referencing the **docker-compose.yaml** file is run.

In [3]:
# Note: Remove Docker containers in case they are still up:
# subprocess.run(["docker", "compose", "down"], shell=False)
# time.sleep(5)

In [4]:
# Note: Build and run Docker containers:
# subprocess.run(["docker", "compose", "up", "-d"], shell=False)
# time.sleep(10)

In [5]:
# Note: Show if containers are up
# subprocess.run(["docker", "container", "ls", "--all"], shell=False)

In [6]:
# Note: RUN THIS IN TERMINAL TO GET LOGS: docker compose logs --follow --since 10m

#### DATA

In [7]:
# Note: Empty cache
torch.cuda.empty_cache()

In [8]:
# Note: Load monthly news articles
dl = DataLoader()
df_demo = dl.load_df(f'src/A_data/monthly/df_demo.parquet')
print('Number of Articles: ', len(df_demo.index))

Number of Articles:  48


In [9]:
# Note: Reduce size
k = 20
# indexes = sorted(choices(population=range(len(df_demo.index)), k=k))
indexes = [0, 2, 5, 12, 15, 16, 21, 23, 23, 24, 25, 25, 29, 41, 42, 43, 43, 44, 46, 47]
df_demo = df_demo.loc[indexes]

#### spaCy NER and COREF

In [10]:
# Note: Run "NER" and "Coreference Resolution" component with spacy pipeline
spacy_process = SpacyProcess(spacy_task=SpacyTask.ALL, ner_method=ExtractionType.TRADITIONAL, 
                             coref_method=ExtractionType.GENERATIVE_LLM)
print('spacy pipeline EN:', spacy_process.nlp_en.pipe_names)
print('spacy pipeline DE:', spacy_process.nlp_de.pipe_names)

CPU is used!
custom extensions "init_extensions" initialized
Built-In "PARSER" initialized
regex_entity_pattern for own_regex_search were compiled.
Function "own_regex_search" initialized
Function "own_coref_resolve_generative" initialized
custom extensions "own_sentencizer" initialized
CPU is used!
custom extensions "init_extensions" initialized
Built-In "PARSER" initialized
regex_entity_pattern for own_regex_search were compiled.
Function "own_regex_search" initialized
Function "own_coref_resolve_generative" initialized
custom extensions "own_sentencizer" initialized
spacy pipeline EN: ['own_sentencizer', 'transformer', 'parser', 'own_regex_search', 'llm_coref_resolve']
spacy pipeline DE: ['own_sentencizer', 'tok2vec', 'parser', 'own_regex_search', 'llm_coref_resolve']


In [11]:
# Note: Coreference Resolution component runs in Docker container
df_after_spacy = spacy_process.run_spacy_pipeline(df=df_demo)

search_match in get_search_match_instance: SearchMatch(comp_name='Zalando SE', comp_symbol='ZAL.DE', text='Zalando', label='OWN-REGEX', start_idx=46, end_idx=53, idx_refer_to=<IDXReferTo.CHARS: 'chars'>)
search_matches in get_matches_from_regex_search: [SearchMatch(comp_name='Zalando SE', comp_symbol='ZAL.DE', text='Zalando', label='OWN-REGEX', start_idx=46, end_idx=53, idx_refer_to=<IDXReferTo.CHARS: 'chars'>)]
#########################
ents_with_cust_exts: [EntsWithCustExts(start_char=46, end_char=53, ent_text='Zalando', comp_name='Zalando SE', comp_symbol='ZAL.DE', set_in='own_regex_search', df_index=None)]
############################
search_matches in get_matches_from_regex_search: []
search_match in get_search_match_instance: SearchMatch(comp_name='Commerzbank AG', comp_symbol='CBK.DE', text='Commerzbank', label='OWN-REGEX', start_idx=48, end_idx=59, idx_refer_to=<IDXReferTo.CHARS: 'chars'>)
search_matches in get_matches_from_regex_search: [SearchMatch(comp_name='Commerzbank AG',

In [12]:
print('df columns:', df_after_spacy.columns)
df_after_spacy

df columns: Index(['art_source', 'art_url', 'art_type', 'art_datetime', 'art_language',
       'art_isin', 'art_company_name', 'art_title', 'art_text', 'art_author',
       'art_abstract', 'art_video_url', 'pp_art_text', 'ner_coref'],
      dtype='object')


,art_source,art_url,art_type,art_datetime,art_language,art_isin,art_company_name,art_title,art_text,art_author,art_abstract,art_video_url,pp_art_text,ner_coref
0,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 10:54:00+02:00,DE,DE000STAB1L8,Stabilus SE,AKTIE IM FOKUS: Stabilus verlieren - Anleger u...,FRANKFURT (dpa-AFX) - Die Ergebnisse von Stabi...,<NA>,<NA>,<NA>,Die Ergebnisse von Stabilus fuer das zweite Ge...,[{'sentence': 'Akshat Kacker von JPMorgan geht...
2,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 11:39:00+02:00,DE,DE0007235301,SGL Carbon SE,ANALYSE/Deutsche Bank: Umstrukturierung von SG...,FRANKFURT (dpa-AFX) - Der Kohlenstoffspezialis...,<NA>,<NA>,<NA>,Der Kohlenstoffspezialist SGL Carbon duerfte n...,[{'sentence': 'Der Kohlenstoffspezialist SGL C...
5,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 10:56:00+02:00,DE,DE000A1MME74,Netfonds AG,EQS-News: Netfonds AG: Erwerb eigener Aktien -...,Netfonds AG: Erwerb eigener Aktien - 24. Zwisc...,<NA>,<NA>,<NA>,Netfonds AG: Erwerb eigener Aktien - 24. Zwisc...,[{'sentence': 'Netfonds AG: Erwerb eigener Akt...
12,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-04 16:35:00+02:00,DE,DE000ZAL1111,Zalando SE,ANALYSE-FLASH: RBC senkt Ziel für Zalando auf ...,NEW YORK (dpa-AFX Broker) - Die kanadische Ban...,<NA>,<NA>,<NA>,Die kanadische Bank RBC hat das Kursziel fuer ...,[{'sentence': 'Die kanadische Bank RBC hat das...
15,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-05 11:31:00+02:00,DE,CH0009236461,Basler Kantonalbank,EQS-News: Das Private Banking der BKB wird aus...,Das Private Banking der BKB wird ausgezeichnet...,<NA>,<NA>,<NA>,Das Private Banking der BKB wird ausgezeichnet...,[]
16,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-08 15:57:00+02:00,DE,DE0006599905,Merck KGaA,AKTIE IM FOKUS: Finanzchef-Rücktritt bei Merck...,FRANKFURT (dpa-AFX) - Der Rücktritt von Finanz...,<NA>,<NA>,<NA>,Der Ruecktritt von Finanzvorstand Marcus Kuhne...,[{'sentence': 'Der Ruecktritt von Finanzvorsta...
21,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-09 09:00:00+02:00,DE,DE0007856023,ElringKlinger AG,ElringKlinger startet besser ins neue Geschäft...,DETTINGEN/ERMS (dpa-AFX) - Der Autozulieferer ...,<NA>,<NA>,<NA>,DETTINGEN ERMS - Der Autozulieferer ElringKlin...,[{'sentence': 'DETTINGEN ERMS - Der Autozulief...
23,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-10 19:56:00+02:00,DE,DE0008303504,TAG Immobilien AG,EQS-Adhoc: TAG Immobilien AG: Ausscheiden von ...,TAG Immobilien AG: Ausscheiden von Herrn Rolf ...,<NA>,<NA>,<NA>,TAG Immobilien AG: Ausscheiden von Herrn Rolf ...,[{'sentence': 'TAG Immobilien AG: Ausscheiden ...
23,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-10 19:56:00+02:00,DE,DE0008303504,TAG Immobilien AG,EQS-Adhoc: TAG Immobilien AG: Ausscheiden von ...,TAG Immobilien AG: Ausscheiden von Herrn Rolf ...,<NA>,<NA>,<NA>,TAG Immobilien AG: Ausscheiden von Herrn Rolf ...,[{'sentence': 'TAG Immobilien AG: Ausscheiden ...
24,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-10 15:03:00+02:00,DE,DE0005493092,Borussia Dortmund GmbH & Co. KGaA,SPORT/Berichte: Borussia Dortmund muss auf Byn...,DORTMUND (dpa-AFX) - Borussia Dortmund muss im...,<NA>,<NA>,<NA>,Borussia Dortmund muss im Endspurt um die deut...,[{'sentence': 'Borussia Dortmund muss im Endsp...


In [13]:
df_after_spacy.ner_coref.loc[0]

[{'sentence': 'Akshat Kacker von JPMorgan geht davon aus, dass das Erreichen des Gewinnausblicks einmal mehr vom spaeteren Verlauf des Geschaeftsjahres abhaengt.',
  'entities': [{'start_char': 784,
    'end_char': 792,
    'ent_text': 'JPMorgan',
    'comp_name': 'JPMorgan Global Core Real Assets Limited',
    'comp_symbol': 'JARA.L',
    'set_in': 'own_regex_search',
    'df_index': None}]}]

#### CONVERT NESTED NER_COREF DICTIONARY

In [15]:
# Note: NER and Coreference Resolution extraction in nested dictionary must be flattened
df_after_convert_nested = SpacyProcess.convert_nested_ner_coref_dict(df_after_spacy)
print('Number of Sentences:', len(df_after_convert_nested))
df_after_convert_nested.head(10)

Number of Sentences: 42


,art_source,art_url,art_type,art_datetime,art_language,art_isin,art_company_name,art_title,art_text,art_author,art_abstract,art_video_url,pp_art_text,ner_coref,art_id,top_sent,top_sent_masked
0,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 10:54:00+02:00,DE,DE000STAB1L8,Stabilus SE,AKTIE IM FOKUS: Stabilus verlieren - Anleger u...,FRANKFURT (dpa-AFX) - Die Ergebnisse von Stabi...,<NA>,<NA>,<NA>,Die Ergebnisse von Stabilus fuer das zweite Ge...,{'sentence': 'Akshat Kacker von JPMorgan geht ...,0,"Akshat Kacker von JPMorgan geht davon aus, das...",Akshat Kacker von Comp@Name@Placeholder geht d...
1,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 11:39:00+02:00,DE,DE0007235301,SGL Carbon SE,ANALYSE/Deutsche Bank: Umstrukturierung von SG...,FRANKFURT (dpa-AFX) - Der Kohlenstoffspezialis...,<NA>,<NA>,<NA>,Der Kohlenstoffspezialist SGL Carbon duerfte n...,{'sentence': 'Der Kohlenstoffspezialist SGL Ca...,2,Der Kohlenstoffspezialist SGL Carbon duerfte n...,Der Kohlenstoffspezialist Comp@Name@Placeholde...
2,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 11:39:00+02:00,DE,DE0007235301,SGL Carbon SE,ANALYSE/Deutsche Bank: Umstrukturierung von SG...,FRANKFURT (dpa-AFX) - Der Kohlenstoffspezialis...,<NA>,<NA>,<NA>,Der Kohlenstoffspezialist SGL Carbon duerfte n...,{'sentence': 'Mit der Einstufung Buy empfiehlt...,2,Mit der Einstufung Buy empfiehlt die Deutsche ...,Mit der Einstufung Buy empfiehlt die Comp@Name...
3,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 10:56:00+02:00,DE,DE000A1MME74,Netfonds AG,EQS-News: Netfonds AG: Erwerb eigener Aktien -...,Netfonds AG: Erwerb eigener Aktien - 24. Zwisc...,<NA>,<NA>,<NA>,Netfonds AG: Erwerb eigener Aktien - 24. Zwisc...,{'sentence': 'Netfonds AG: Erwerb eigener Akti...,5,Netfonds AG: Erwerb eigener Aktien - 24. Zwisc...,Comp@Name@Placeholder AG: Erwerb eigener Aktie...
4,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 10:56:00+02:00,DE,DE000A1MME74,Netfonds AG,EQS-News: Netfonds AG: Erwerb eigener Aktien -...,Netfonds AG: Erwerb eigener Aktien - 24. Zwisc...,<NA>,<NA>,<NA>,Netfonds AG: Erwerb eigener Aktien - 24. Zwisc...,{'sentence': 'Netfonds AG: Erwerb eigener Akti...,5,Netfonds AG: Erwerb eigener Aktien - 24. Zwisc...,Comp@Name@Placeholder AG: Erwerb eigener Aktie...
5,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 10:56:00+02:00,DE,DE000A1MME74,Netfonds AG,EQS-News: Netfonds AG: Erwerb eigener Aktien -...,Netfonds AG: Erwerb eigener Aktien - 24. Zwisc...,<NA>,<NA>,<NA>,Netfonds AG: Erwerb eigener Aktien - 24. Zwisc...,{'sentence': 'Im Zeitraum vom 24. April 2023 b...,5,Im Zeitraum vom 24. April 2023 bis einschliess...,Im Zeitraum vom 24. April 2023 bis einschliess...
6,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 10:56:00+02:00,DE,DE000A1MME74,Netfonds AG,EQS-News: Netfonds AG: Erwerb eigener Aktien -...,Netfonds AG: Erwerb eigener Aktien - 24. Zwisc...,<NA>,<NA>,<NA>,Netfonds AG: Erwerb eigener Aktien - 24. Zwisc...,{'sentence': 'Der Erwerb der Aktien der Netfon...,5,Der Erwerb der Aktien der Netfonds AG erfolgt ...,Der Erwerb der Aktien der Comp@Name@Placeholde...
7,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 10:56:00+02:00,DE,DE000A1MME74,Netfonds AG,EQS-News: Netfonds AG: Erwerb eigener Aktien -...,Netfonds AG: Erwerb eigener Aktien - 24. Zwisc...,<NA>,<NA>,<NA>,Netfonds AG: Erwerb eigener Aktien - 24. Zwisc...,{'sentence': 'Delegierte Verordnung Nr. 2016 1...,5,Delegierte Verordnung Nr. 2016 1052 sind auf d...,Delegierte Verordnung Nr. 2016 1052 sind auf d...
8,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-04 16:35:00+02:00,DE,DE000ZAL1111,Zalando SE,ANALYSE-FLASH: RBC senkt Ziel für Zalando auf ...,NEW YORK (dpa-AFX Broker) - Die kanadische Ban...,<NA>,<NA>,<NA>,Die kanadische Ban

In [15]:
# df_after_convert_nested.to_parquet('/media/rainergo/PROJECTS/UASFRA-MS-Thesis/src/A_data/monthly/df_demo_after_ner_coref.parquet')

#### START TOPIC MODELING

##### TOPIC MODELLING with GEN-LLMs

In [16]:
process = Process()

In [17]:
# Note: Topic Modelling component runs in Docker container
df_after_topics = process.get_topics_from_gen_llm(df=df_after_convert_nested, chunk_size=20)

df_chunk.index.tolist(): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
frame.indexes [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
Error occurred. Will try again... 
Error occured the second time, could not be cured: Internal Server Error
DEBUG INFO: ['topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17']
df_chunk.index.tolist(): [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
frame.indexes [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
Error occurred. Will try again... 
Error occured the second time, could not be cured: Internal Server Error
DEBUG INFO: ['topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 'topic17', 't

ValueError: Length of values (60) does not match length of index (42)

In [18]:
df_after_topics

,art_source,art_url,art_type,art_datetime,art_language,art_isin,art_company_name,art_title,art_text,art_author,art_abstract,art_video_url,pp_art_text,ner_coref,art_id,top_sent,top_sent_masked,topic
0,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 10:54:00+02:00,DE,DE000STAB1L8,Stabilus SE,AKTIE IM FOKUS: Stabilus verlieren - Anleger u...,FRANKFURT (dpa-AFX) - Die Ergebnisse von Stabi...,<NA>,<NA>,<NA>,Die Ergebnisse von Stabilus fuer das zweite Ge...,{'sentence': 'Akshat Kacker von JPMorgan geht ...,0,"Akshat Kacker von JPMorgan geht davon aus, das...",Akshat Kacker von Comp@Name@Placeholder geht d...,topic14
1,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 10:54:00+02:00,DE,DE000STAB1L8,Stabilus SE,AKTIE IM FOKUS: Stabilus verlieren - Anleger u...,FRANKFURT (dpa-AFX) - Die Ergebnisse von Stabi...,<NA>,<NA>,<NA>,Die Ergebnisse von Stabilus fuer das zweite Ge...,{'sentence': 'Die Markterwartungen laegen aber...,0,Die Markterwartungen laegen aber bereits am ob...,Die Markterwartungen laegen aber bereits am ob...,topic14
2,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 11:39:00+02:00,DE,DE0007235301,SGL Carbon SE,ANALYSE/Deutsche Bank: Umstrukturierung von SG...,FRANKFURT (dpa-AFX) - Der Kohlenstoffspezialis...,<NA>,<NA>,<NA>,Der Kohlenstoffspezialist SGL Carbon duerfte n...,{'sentence': 'Der Kohlenstoffspezialist SGL Ca...,2,Der Kohlenstoffspezialist SGL Carbon duerfte n...,Der Kohlenstoffspezialist Comp@Name@Placeholde...,topic14
3,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 11:39:00+02:00,DE,DE0007235301,SGL Carbon SE,ANALYSE/Deutsche Bank: Umstrukturierung von SG...,FRANKFURT (dpa-AFX) - Der Kohlenstoffspezialis...,<NA>,<NA>,<NA>,Der Kohlenstoffspezialist SGL Carbon duerfte n...,{'sentence': 'Vom-Cleff haelt die 2023er-Progn...,2,Vom-Cleff haelt die 2023er-Prognose des Untern...,Vom-Cleff haelt die 2023er-Prognose des Comp@N...,topic14
4,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 11:39:00+02:00,DE,DE0007235301,SGL Carbon SE,ANALYSE/Deutsche Bank: Umstrukturierung von SG...,FRANKFURT (dpa-AFX) - Der Kohlenstoffspezialis...,<NA>,<NA>,<NA>,Der Kohlenstoffspezialist SGL Carbon duerfte n...,{'sentence': 'Er beruecksichtigt dabei die Unt...,2,Er beruecksichtigt dabei die Unternehmensziele...,Er beruecksichtigt dabei die Unternehmensziele...,topic4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-18 09:19:00+02:00,DE,GB00B7KR2P84,Easyjet PLC,Easyjet rechnet mit starkem Sommer - auch dank...,LUTON (dpa-AFX) - Der britische Billigflieger ...,<NA>,<NA>,<NA>,LUTON - Der britische Billigflieger Easyjet pr...,{'sentence': 'Im laufenden Geschaeftsjahr duer...,46,Im laufenden Geschaeftsjahr duerfte der berein...,Im laufenden Geschaeftsjahr duerfte der berein...,topic4
67,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-18 10:38:00+02:00,DE,GB00BDVZYZ77,International Distributions SE,Royal Mail gibt Streiks Schuld an Milliardenve...,LONDON (dpa-AFX) - Der britische Postdienstlei...,<NA>,<NA>,<NA>,Der britische Postdienstleister Royal Mail hat...,{'sentence': 'Insgesamt machte das Unternehmen...,47,Insgesamt machte das Unternehmen im Geschaefts...,Insgesamt machte das Unternehmen im Geschaefts...,topic1
68,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-18 10:38:00+02:00,DE,GB00BDVZYZ77,International Distributions SE,Royal Mail gibt Streiks Schuld an Milliardenve...,LONDON (dpa-AFX) - Der britische Postdienstlei...,<NA>,<NA>,<NA>,Der britische Postdienstleister Royal Mail hat...,{'sentence': 'Die Verluste seien vor allem auf...,47,Die Verluste seien vor allem auf die Auswirkun...,Die Verluste seien vor allem auf die Auswirkun...,topic13
69,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,

In [19]:
# Note: pandas DataFrame must be adjusted
df_after_prepare = process.prepare_df_for_kg(df=df_after_topics)
df_after_prepare

,index,art_source,art_url,art_type,art_datetime,art_language,art_isin,art_company_name,art_title,art_text,art_author,art_abstract,art_video_url,pp_art_text,ner_coref,art_id,top_sent,top_sent_masked,topic,ner_coref_entities,comp_symbol,comp_name,top_description,sent_id,comp_symbol_raw,comp_symbol_country,comp_isin
0,0,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 10:54:00+02:00,DE,DE000STAB1L8,Stabilus SE,AKTIE IM FOKUS: Stabilus verlieren - Anleger u...,FRANKFURT (dpa-AFX) - Die Ergebnisse von Stabi...,<NA>,<NA>,<NA>,Die Ergebnisse von Stabilus fuer das zweite Ge...,{'sentence': 'Akshat Kacker von JPMorgan geht ...,0,"Akshat Kacker von JPMorgan geht davon aus, das...",Akshat Kacker von Comp@Name@Placeholder geht d...,topic14,"{'start_char': 784, 'end_char': 792, 'ent_text...",JARA.L,JPMorgan Global Core Real Assets Limited,Einschätzungen Unternehmensfremder/Analysten z...,0,JARA,L,isin_not_found
1,1,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 10:54:00+02:00,DE,DE000STAB1L8,Stabilus SE,AKTIE IM FOKUS: Stabilus verlieren - Anleger u...,FRANKFURT (dpa-AFX) - Die Ergebnisse von Stabi...,<NA>,<NA>,<NA>,Die Ergebnisse von Stabilus fuer das zweite Ge...,{'sentence': 'Die Markterwartungen laegen aber...,0,Die Markterwartungen laegen aber bereits am ob...,Die Markterwartungen laegen aber bereits am ob...,topic14,"{'start_char': 1101, 'end_char': 1117, 'ent_te...",JARA.L,JPMorgan Global Core Real Assets Limited,Einschätzungen Unternehmensfremder/Analysten z...,1,JARA,L,isin_not_found
2,2,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 11:39:00+02:00,DE,DE0007235301,SGL Carbon SE,ANALYSE/Deutsche Bank: Umstrukturierung von SG...,FRANKFURT (dpa-AFX) - Der Kohlenstoffspezialis...,<NA>,<NA>,<NA>,Der Kohlenstoffspezialist SGL Carbon duerfte n...,{'sentence': 'Der Kohlenstoffspezialist SGL Ca...,2,Der Kohlenstoffspezialist SGL Carbon duerfte n...,Der Kohlenstoffspezialist Comp@Name@Placeholde...,topic14,"{'start_char': 26, 'end_char': 36, 'ent_text':...",SGL.DE,SGL Carbon SE,Einschätzungen Unternehmensfremder/Analysten z...,2,SGL,DE,DE0007235301
3,3,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 11:39:00+02:00,DE,DE0007235301,SGL Carbon SE,ANALYSE/Deutsche Bank: Umstrukturierung von SG...,FRANKFURT (dpa-AFX) - Der Kohlenstoffspezialis...,<NA>,<NA>,<NA>,Der Kohlenstoffspezialist SGL Carbon duerfte n...,{'sentence': 'Vom-Cleff haelt die 2023er-Progn...,2,Vom-Cleff haelt die 2023er-Prognose des Untern...,Vom-Cleff haelt die 2023er-Prognose des Comp@N...,topic14,"{'start_char': 390, 'end_char': 402, 'ent_text...",SGL.DE,SGL Carbon SE,Einschätzungen Unternehmensfremder/Analysten z...,3,SGL,DE,DE0007235301
4,4,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-02 11:39:00+02:00,DE,DE0007235301,SGL Carbon SE,ANALYSE/Deutsche Bank: Umstrukturierung von SG...,FRANKFURT (dpa-AFX) - Der Kohlenstoffspezialis...,<NA>,<NA>,<NA>,Der Kohlenstoffspezialist SGL Carbon duerfte n...,{'sentence': 'Er beruecksichtigt dabei die Unt...,2,Er beruecksichtigt dabei die Unternehmensziele...,Er beruecksichtigt dabei die Unternehmensziele...,topic4,"{'start_char': 715, 'end_char': 718, 'ent_text...",SGL.DE,SGL Carbon SE,"Zukunftsgerichteter Ausblick, Prognosen, Ziele...",4,SGL,DE,DE0007235301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,77,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-18 09:19:00+02:00,DE,GB00B7KR2P84,Easyjet PLC,Easyjet rechnet mit starkem Sommer - auch dank...,LUTON (dpa-AFX) - Der britische Billigflieger ...,<NA>,<NA>,<NA>,LUTON - Der britische Billigflieger Easyjet pr...,{'sentence': 'Im laufenden Geschaeftsjahr duer...,46,Im laufenden Geschaeftsjahr duerfte der berein...,Im laufenden Geschaeftsjahr duerfte der berein...,topic4,"{'start_char': 851, 'end_char': 858, 'ent_text...",EJT1.DE,e

In [20]:
df_after_prepare[['art_id', 'topic', 'top_sent', 'top_description']]

,art_id,topic,top_sent,top_description
0,0,topic14,"Akshat Kacker von JPMorgan geht davon aus, das...",Einschätzungen Unternehmensfremder/Analysten z...
1,0,topic14,Die Markterwartungen laegen aber bereits am ob...,Einschätzungen Unternehmensfremder/Analysten z...
2,2,topic14,Der Kohlenstoffspezialist SGL Carbon duerfte n...,Einschätzungen Unternehmensfremder/Analysten z...
3,2,topic14,Vom-Cleff haelt die 2023er-Prognose des Untern...,Einschätzungen Unternehmensfremder/Analysten z...
4,2,topic4,Er beruecksichtigt dabei die Unternehmensziele...,"Zukunftsgerichteter Ausblick, Prognosen, Ziele..."
...,...,...,...,...
58,46,topic4,Im laufenden Geschaeftsjahr duerfte der berein...,"Zukunftsgerichteter Ausblick, Prognosen, Ziele..."
59,47,topic1,Insgesamt machte das Unternehmen im Geschaefts...,Sätze mit konkreten Zahlenangaben aus Quartals...
60,47,topic13,Die Verluste seien vor allem auf die Auswirkun...,Einflüsse von Aussen auf die Erfolgsaussichten...
61,47,topic11,Auch hier spielten laut IDS die Streiks eine R...,"Personalveränderungen im Vorstand, Aufsichtsra..."


In [21]:
# Note: Show sentences and how they were classified topic-wise
for idx, row in df_after_prepare.iterrows():
    print('text:', row.top_sent)
    print('topic:', row.top_description)
    print('-----------------------------')

text: Akshat Kacker von JPMorgan geht davon aus, dass das Erreichen des Gewinnausblicks einmal mehr vom spaeteren Verlauf des Geschaeftsjahres abhaengt.
topic: Einschätzungen Unternehmensfremder/Analysten zu einem Unternehmen
-----------------------------
text: Die Markterwartungen laegen aber bereits am oberen Ende, so der JPMorgan-Experte.
topic: Einschätzungen Unternehmensfremder/Analysten zu einem Unternehmen
-----------------------------
text: Der Kohlenstoffspezialist SGL Carbon duerfte nach Einschaetzung der Deutschen Bank allmaehlich die Fruechte seiner Umstrukturierung ernten.
topic: Einschätzungen Unternehmensfremder/Analysten zu einem Unternehmen
-----------------------------
text: Vom-Cleff haelt die 2023er-Prognose des Unternehmens fuer das operative Ergebnis fuer konservativ, auch der angepeilte Umsatz lediglich auf stabilem Niveau verlange nach Erklaerungen.
topic: Einschätzungen Unternehmensfremder/Analysten zu einem Unternehmen
-----------------------------
text: Er be

#### START KNOWLEDGE GRAPH

##### Initialize Knowledge Graph

In [22]:
# Note: Erase existing Knowledge Graph and initialize settings
from src.F_knowledge_graph.B_graph_construction import GraphConstruction
kg = GraphConstruction()
kg.delete_graph()

In [23]:
kg.init_graph(handle_vocab_uris="IGNORE", handle_mult_vals="OVERWRITE")

In [24]:
# Note: Load data
kg.load_data_into_knowledge_graph(df=df_after_prepare)

In [25]:
# Note: Import wikidata id
kg.import_wikidata_id(node='Company', node_prop='comp_isin', node_prop_wikidata_id='P946')

In [26]:
# Note: Enrich Knowledge Graph with Wikidata and DBPedia Data
# Note: WikiData: Insert "industries" and "countries"
industry = {"node_label": "Company",
            "prop_name": "comp_isin",
            "prop_wiki_id": "P946",
            "new_prop_name": "industries",
            "new_prop_wiki_id": "P452",
            "use_new_prop_label": True,
            "new_prop_is_list": True,
            "prop_wiki_id_is_part_of": None
}
country = {"node_label": "Company",
            "prop_name": "comp_isin",
            "prop_wiki_id": "P946",
           "new_prop_name": "country",
           "new_prop_wiki_id": "P17",
           "use_new_prop_label": True,
           "new_prop_is_list": False,
"prop_wiki_id_is_part_of": None}

wiki_data = [industry, country]
for d in wiki_data:
    kg.import_data_from_wikidata(**d)


        MATCH (node:Company)
        WITH 
            "SELECT ?comp_isin ?industriesLabel
                WHERE {
                    FILTER (?comp_isin = \"" + node.comp_isin + "\")
                    ?company wdt:P946 ?comp_isin ;
                             wdt:P452 ?industries .
                SERVICE wikibase:label { bd:serviceParam wikibase:language \"en\". }
              }"     
        AS sparql
        CALL apoc.load.jsonParams(
            "https://query.wikidata.org/sparql?query=" + 
              apoc.text.urlencode(sparql),
            { Accept: "application/sparql-results+json"}, null)
        YIELD value
        UNWIND value['results']['bindings'] as row
        WITH row['comp_isin']['value'] as prop_val, 
             collect(row['industriesLabel']['value']) as new_prop_val
        MERGE (n:Company { comp_isin: prop_val })
        SET n.industries = new_prop_val;
        

        MATCH (node:Company)
        WITH 
            "SELECT ?comp_isin ?countryLabel
    

In [27]:
# Note: Import "Company Abstract" from DBPedia
node_label = "Company"
prop_name = "wikidataID"
prop_dbp_id = "owl:sameAs"
new_prop_name = "abstract"
new_prop_dbp_id = "dbo:abstract"
new_prop_is_list: bool = False
kg.import_data_from_dbpedia(node_label=node_label, prop_name=prop_name, prop_dbp_id=prop_dbp_id,
                            new_prop_name=new_prop_name, new_prop_dbp_id=new_prop_dbp_id,
                            new_prop_is_list=new_prop_is_list)


        MATCH (node:Company)
        WITH
            "PREFIX wd: <http://www.wikidata.org/entity/>
             SELECT ?wikidataID ?abstract
                    WHERE {
                        BIND(<"+node.wikidataID+"> AS ?wikidataID)
                        ?dbpcomp   owl:sameAs   ?wikidataID ;
                                   dbo:abstract ?abstract .
                    FILTER langMatches( lang(?abstract), \"en\" )
                  }
                  LIMIT 1
                  "   
        AS sparql
        CALL apoc.load.jsonParams(
            "https://dbpedia.org/sparql?query=" + 
              apoc.text.urlencode(sparql),
            { Accept: "application/sparql-results+json"}, null)
        YIELD value
        UNWIND value['results']['bindings'] as row
        WITH row['wikidataID']['value'] as prop_val,
             row['abstract']['value'] as new_prop_val
        MERGE (n:Company { wikidataID: prop_val })
        SET n.abstract = new_prop_val;
        


In [ ]:
# Note: Create text embeddings for news article sentences
kg.create_text_embedding(node_label="Sentence", node_primary_prop_name="sent_id", 
                         prop_to_embed="sent_text")

In [18]:
# Note: Communicate with Graph Bot
from src.F_knowledge_graph.D_graph_bot import GraphBot
qa = GraphBot()
# print(qa.get_schema())
question = "Show me all the companies that were mentioned of articles that were published between 2023-05-12 and 2023-05-17"
# question = "Was the company 'Vonovia SE' or the company 'Bechtle AG' mentioned in article sentences, and if yes, what was the sentence text all about?"
# question = "For each company that is located in Germany, get each article sentence text and its topic only for date 2023-05-17. Make sure to separate your answer by company name."
print('QUESTION:\n', question)
# print('Answer:\n')
ans = qa.ask_question(question=question)
print('ANSWER:\n', ans)

QUESTION:
 Show me all the companies that were mentioned of articles that were published between 2023-05-12 and 2023-05-17


> Entering new GraphCypherQAChain chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

#### CLEAN UP / REMOVE DOCKER CONTAINER

In [ ]:
# Note: Stop Docker containers and clean up to free memory
# subprocess.run(["docker", "compose", "down"], shell=False)
# torch.cuda.empty_cache()